In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/16 14:27:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/16 14:27:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/16 14:27:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/03/16 14:27:32 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


#  Group By in Spark

## Groupby green data 

In [3]:
df_green = spark.read.parquet('../code/data/pq/green/*/*')

In [4]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [8]:
# make a temporary SQL table

df_green.createOrReplaceTempView('green_data')

In [22]:
# create a new query

df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records

FROM
    green_data
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [16]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999997|            45|
|2020-01-01 00:00:00|  17|195.03000000000003|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25| 531.0000000000001|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|            317.27|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|            295.34|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [35]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('../code/data/report/revenue/green', mode='overwrite')

## Groupby yellow data 

In [25]:
df_yellow = spark.read.parquet('../code/data/pq/yellow/*/*')

In [26]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [29]:
# make a temporary SQL table

df_yellow.createOrReplaceTempView('yellow_data')

In [32]:
# create a new query

df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records

FROM
    yellow_data
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [33]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 08:00:00| 162| 767.8100000000003|            46|
|2020-01-04 18:00:00| 140|2618.9099999999985|           169|
|2020-01-27 16:00:00|  88|1478.9999999999995|            53|
|2020-01-05 02:00:00| 114| 5647.420000000002|           329|
|2020-01-19 23:00:00|  90|2217.8799999999987|           150|
|2020-01-23 17:00:00| 140| 6545.520000000008|           364|
|2020-01-09 11:00:00| 107|3677.2599999999966|           240|
|2020-01-26 20:00:00| 239|2532.9399999999987|           178|
|2020-01-03 23:00:00| 132| 24519.23999999997|           448|
|2020-01-26 20:00:00| 236|3540.5199999999977|           242|
|2020-01-24 19:00:00| 186|11532.320000000029|           625|
|2020-01-24 03:00:00| 264|             249.6|            11|
|2020-01-31 15:00:00| 143|2763.2299999999977|           182|
|2020-01-23 21:00:00| 14

In [36]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('../code/data/report/revenue/yellow', mode='overwrite')

# Joins in Spark 

In [37]:
df_green_revenue = spark.read.parquet('../code/data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('../code/data/report/revenue/yellow')

In [39]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [40]:
# outer join means if there is no record in either dataframe for specific hour and zone, it will be filled with null
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [42]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999997|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  13|              null|                null|1214.8000000000002|                   56|
|2020-01-01 00:00:00|  18|               7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  25| 531.0000000000001|                  26|            324.35|                   16|
|2020-01-01 00:00:00|  32| 68.94999999999999|                   2|              18.0|                    1|
|2020-01-01 00:00:00|  34|  

In [43]:
df_join.write.parquet('../code/data/report/revenue/total', mode='overwrite')

## Joining large dataset and small dataset  

In [44]:
df_join = spark.read.parquet('../code/data/report/revenue/total/')

In [45]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  10|              null|                null|             42.41|                    2|
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  24|              87.6|                   3|            754.95|                   45|
|2020-01-01 00:00:00|  29|              61.3|                   1|              null|                 null|
|2020-01-01 00:00:00|  33|  

In [46]:
df_zones = spark.read.parquet('zones/')

In [47]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [51]:
# joining df_join and df_zones

df_result = df_join.join(df_zones, df_join['zone'] == df_zones['LocationID'])

In [56]:
df_result.drop('LocationID', 'zone').show()

+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|2020-01-01 00:00:00|              null|                null|             42.41|                    2|   Queens|   Boro Zone|
|2020-01-01 00:00:00|              null|                null|               8.8|                    1| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|              null|                null|             34.09|                    1|   Queens|   Boro Zone|
|2020-01-01 00:00:00|195.03000000000003|                   9|220.20999999999998|                    8| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|              87.6|                   3|            754.95|                   45|Manhattan| Yello

In [57]:
df_result.drop('LocationID', 'zone').write.parquet('../code/data/revenue_zones/')